# Introduction 

This jupyter notbook is to illustrate the process of extracting survival curve from published literature in order to generate parametric/non-parametric survival distrbutions for use in cost-effectiveness analysis (CEA) or health technology assessments (HTA).

# System Information

In [1]:
import sys
print (sys.version)

3.6.1 (default, Sep  7 2017, 16:36:03) 
[GCC 6.3.0 20170406]


The next line is to allow for printing all python code as opposed to just the last line in the code syntax

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#  Example Curve

The curve that we will be using will be from [Neelapu et al](http://www.nejm.org/doi/full/10.1056/NEJMoa1707447) where the survival plot figure 2B indicates progression free surivival probabilities (from Kaplan-Meier estimates) are plotted. 

Below is a very brief tutorial on how to go about extracting the curve using the [WebPlotDigitizer](https://apps.automeris.io/wpd/). This app is very useful but requires a bit of manual handling (for now... working on a script that will generate the curves automatically).

# Curve Extraction using WebPlotDigitizer

1. Using snipping tool, extract a png file from the original pdf document of Figure 2B of the article and and input into the application using File -> Load image.

![title](/srv/shiny-server/SurvivalCurveExtraction/www/step1.gif)